In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

## Dates & Other Constants

In [2]:
PATH = "Stock Data/Collection of Stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-10-01"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

## Get Stock File Names in a List

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]

tickers.sort()

## Function that Returns a Dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

## Function that Saves Dataframe to CSV

In [5]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

## Return 1st Valid Date in Dataframe

In [6]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

## Returns Return on Investment over Time

In [7]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

## Get Mean Between Dates

In [8]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

## Get Standard Deviation Between Dates

In [9]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

## Get Coefficient of Variation Between Dates

In [10]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

## Merge Multiple Stocks in One Dataframe by Column Name

In [11]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

## 11 Stock Market Sectors

**Technology** : Manufacturing of electronics, software, or related to information technology 

**Health Care** : Biotech, hospitals, medical devices, drugs 

**Financials** : Banks, investment funds, and insurance

**Real Estate** : Residential, industrial, and retail real estate 

**Energy** : Production and supply of energy 

**Materials** : Mining, refining, chemical, and forestry 

**Consumer Discretionary** : Retailers, apparel, media, durables and services 

**Industrials** : Construction, machinery, fabrication, manufacturing, defense, and aerospace 

**Utilities** : Direct providers of electric, gas, and water 

**Consumer Staples** : Food, beverage as well as products consumers deem essential 

**Telecommunication** : Cable, internet providers, wireless, and satellite

## Get Sector Stocks

In [12]:
# For Windows
sec_df = pd.read_csv("Stock Data/Collection of Stocks/stock_sectors.csv")


# Get Industrials DF
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

## Returns a DF with ROIs for all Stocks

In [13]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            pass
            # print(row['Symbol'], " is not available")
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [14]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

## Top Industrials

In [15]:
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
24,GNRC,3.734749
10,CPRT,1.918578
57,TDG,1.518984
9,CTAS,1.325593
42,ODFL,1.202971


## Top Healthcare

In [16]:
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
25,DXCM,5.375582
33,IDXX,2.167543
59,WST,1.887307
49,RMD,1.575014
16,CTLT,1.475499


## Top Information Tech

In [17]:
it.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
21,ENPH,66.488464
2,AMD,7.352459
50,PAYC,4.622203
57,NOW,3.178471
70,ZBRA,2.705814


## Top Communications

In [18]:
commun.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,ROI
15,NFLX,1.689262
22,TWTR,1.209302
19,TMUS,1.105715
4,CHTR,0.895722
20,TTWO,0.840804
14,LYV,0.722861
21,DIS,0.666390
2,GOOG,0.644958
1,GOOGL,0.633082
10,FB,0.505677


## Top Staples

In [19]:
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,EL,1.145196
9,COST,1.112863
19,MKC,0.977708
3,CHD,0.838448
25,PG,0.674050


## Top Discretionary

In [20]:
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
52,TSLA,10.007861
17,ETSY,7.541047
10,CMG,3.733606
43,POOL,1.931030
51,TGT,1.839621


## Top Utilities

In [21]:
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,AES,1.402913
18,NEE,1.136333
4,AWK,0.819223
26,WEC,0.530405
27,XEL,0.519897


## Top Finance

In [22]:
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
39,MSCI,2.612068
34,MKTX,1.860333
6,AJG,1.112315
37,MCO,1.044882
49,SPGI,1.012123


## Top Materials

In [23]:
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
3,BLL,1.474667
22,SHW,0.824243
14,LIN,0.785525
0,APD,0.771644
17,NEM,0.689577


## Top Real Estate

In [24]:
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
21,SBAC,0.772695
8,EQIX,0.692928
17,PLD,0.689758
1,AMT,0.685782
5,CCI,0.622156


## Top Energy

In [25]:
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
9,HES,0.168440
15,OKE,-0.131482
10,KMI,-0.146362
3,COP,-0.217487
20,WMB,-0.218709


## Portfolio List

In [27]:
port_list = ["GNRC" , "DXCM", "ENPH" , "NFLX" , "EL" , "TSLA" , "AES" , "MSCI" , "BLL" , "SBAC" , "HES"]

## Check for Correlation

In [28]:
mult_df = merge_df_by_column_name('daily_return',  '2018-01-01', 
                                  '2020-12-31', *port_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

,GNRC,DXCM,ENPH,NFLX,EL,TSLA,AES,MSCI,BLL,SBAC,HES
GNRC,1.000000,0.289526,0.344129,0.340732,0.418513,0.321049,0.433322,0.515742,0.446344,0.444762,0.426870
DXCM,0.289526,1.000000,0.212156,0.307643,0.263909,0.293396,0.218424,0.469509,0.268963,0.243462,0.243239
ENPH,0.344129,0.212156,1.000000,0.282688,0.256654,0.280869,0.261715,0.341827,0.195593,0.207225,0.299728
NFLX,0.340732,0.307643,0.282688,1.000000,0.293105,0.360827,0.204158,0.449132,0.280443,0.216936,0.217857
EL,0.418513,0.263909,0.256654,0.293105,1.000000,0.277749,0.407448,0.457616,0.446227,0.397506,0.445605
TSLA,0.321049,0.293396,0.280869,0.360827,0.277749,1.000000,0.273606,0.373189,0.243563,0.181884,0.259790
AES,0.433322,0.218424,0.261715,0.204158,0.407448,0.273606,1.000000,0.483399,0.368256,0.333249,0.470412
MSCI,0.515742,0.469509,0.341827,0.449132,0.457616,0.373189,0.483399,1.000000,0.459635,0.498281,0.411647
BLL,0.446344,0.268963,0.195593,0.280443,0.446227,0.243563,0.368256,0.459635,1.000000,0.556609,0.291071
SBAC,0.444762,0.243462,0.207225,0.216936,0.397506,0.181884,0.333249,0.498281,0.556609,1.000000,0.273548


In [29]:
# Get the number of samples
days = len(mult_df.index)
days

756

In [30]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [31]:
# 01-04-2021 is the 1st date in 2021
print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) 
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) 
print("ENPH $", get_stock_price_on_date('ENPH', '2021-01-04')) 
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04'))
print("EL $", get_stock_price_on_date('EL', '2021-01-04')) 
print("TSLA $", get_stock_price_on_date('TSLA', '2021-01-04')) 
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) 
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) 
print("BLL $", get_stock_price_on_date('BLL', '2021-01-04')) 
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) 
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) 



GNRC $ 224.5599975585937
DXCM $ 357.739990234375
ENPH $ 172.24000549316406
NFLX $ 522.8599853515625
EL $ 253.69464111328125
TSLA $ 729.77001953125
AES $ 22.819656372070312
MSCI $ 435.3967590332031
BLL $ 89.25436401367188
SBAC $ 273.9728698730469
HES $ 52.284915924072266


## Returns Portfolio Weight

In [32]:
def get_port_weight(price, total):
    return price / total

In [33]:
# BASED on the hypethetical investments made into the following stocks

gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

Portfolio Covariance : 0.24587314908145463
GNRC Var : 0.43946543607242483
NFLX Var : 0.5477253623043377


## Different Types of Risk

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). Systematic risk 
is caused by unforeseen conditions such as wars, recessions, natural disasters, dramatic changes in consumer actions, etc.

Data tells us that if we make a portfolio made up of approximately 25 stocks that aren't correlated that we can dramatically
lower idiosyncratic risk. That is one reason why people invest in indexes. You can further lower risk by investing in
other countries, bonds and cash.

## Diversifiable Risk

In [34]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [35]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.18046007010984796

## Find Risk that Can't be Diversified

In [36]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.06541307897160667
